# Отток клиентов

## Введение

### Задание

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

### Описание данных

- **RowNumber** — индекс строки в данных
- **CustomerId** — уникальный идентификатор клиента
- **Surname** — фамилия
- **CreditScore** — кредитный рейтинг
- **Geography** — страна проживания
- **Gender** — пол
- **Age** — возраст
- **Tenure** — количество недвижимости у клиента
- **Balance** — баланс на счёте
- **NumOfProducts** — количество продуктов банка, используемых клиентом
- **HasCrCard** — наличие кредитной карты
- **IsActiveMember** — активность клиента
- **EstimatedSalary** — предполагаемая зарплата

Целевой признак
- **Exited** — факт ухода клиента

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Вывод 

- Поле **Tenure** содержит пустые значения 
- Поля **Surname**, **Geography**, **Gender** не числовые
- Предположительно поля **RowNumber**, **CustomerId**, **Surname**, **Geography**, **Gender** не влиют на результат 

## Деление выборки

In [5]:
data_train, data_test = train_test_split(data, test_size=0.4, random_state=12345)

## Кодирование признаков 

Поля Surname, Geography, Gender — являются категориальными признаками и мы решаем задачу классификации, поэтому нам подойдет кодировка OrdinalEncoder

In [6]:
data_train = data_train.reset_index()
data_train = data_train.drop('index', axis = 1)

In [7]:
encoder = OrdinalEncoder()
encoder.fit(data[['Surname', 'Geography', 'Gender']])

def ordinal_encoder(data):
    
    tmp = data[['Surname', 'Geography', 'Gender']]
    tmp =  pd.DataFrame(encoder.transform(tmp), columns=tmp.columns)
    
    print(data[['Surname', 'Geography', 'Gender']].head(10))
    
    print('================================')
    
    print(tmp.head(10))
    
    data[['Surname', 'Geography', 'Gender']] = tmp
    return data

In [8]:
data_train = ordinal_encoder(data_train)

    Surname Geography  Gender
0   Meagher     Spain    Male
1    Moysey    France  Female
2   Forster   Germany    Male
3       Hay    France    Male
4   Lorenzo   Germany  Female
5      Hill     Spain  Female
6  Genovese     Spain    Male
7     Azuka    France  Female
8       Yao   Germany  Female
9    Walker   Germany    Male
   Surname  Geography  Gender
0   1774.0        2.0     1.0
1   1868.0        0.0     0.0
2    918.0        1.0     1.0
3   1138.0        0.0     1.0
4   1594.0        1.0     0.0
5   1177.0        2.0     0.0
6    984.0        2.0     1.0
7    135.0        0.0     0.0
8   2868.0        1.0     0.0
9   2751.0        1.0     1.0


In [9]:
data_train.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,7480,15671987,1774.0,567,2.0,1.0,35,8.0,153137.74,1,1,0,88659.07,0
1,3412,15815628,1868.0,711,0.0,0.0,37,8.0,113899.92,1,0,0,80215.20,0
2,6028,15799494,918.0,850,1.0,1.0,44,3.0,140393.65,2,0,1,186285.52,0
3,1248,15711288,1138.0,512,0.0,1.0,24,6.0,0.00,2,1,0,37654.31,0
4,3717,15699492,1594.0,665,1.0,0.0,27,2.0,147435.96,1,0,0,187508.06,0
5,8742,15762855,1177.0,622,2.0,0.0,23,8.0,0.00,2,1,1,131389.39,0
6,7462,15645571,984.0,596,2.0,1.0,32,4.0,0.00,2,0,1,146504.35,0
7,5107,15682995,135.0,600,0.0,0.0,32,1.0,78535.25,1,1,0,64349.60,0
8,6131,15651144,2868.0,632,1.0,0.0,35,2.0,150561.03,2,0,0,64722.61,0
9,4956,15581525,2751.0,775,1.0,1.0,33,NaN,83501.66,2,1,0,128841.31,0


### Заполнение Tenure

In [10]:
def find_importances(X, Y, header):
    
    model = ExtraTreesClassifier()
    model.fit(X, Y)
    
    result = []
    for feat, importance in zip(header, model.feature_importances_):  
        temp = [feat, importance * 100]
        result.append(temp)

    data = pd.DataFrame(result, columns = ['Feature', 'Importance'])
    return data.sort_values('Importance', ascending = False)
    

In [11]:
def split_x_y(data):
    X = data.drop('Tenure', axis=1)
    Y = data['Tenure']
    return X, Y

In [12]:
tenure_train = data_train[data_train['Tenure'].notna()] 

tenure_train_x, tenure_train_y = split_x_y(tenure_train)

# feature extraction
importances = find_importances(tenure_train_x, tenure_train_y, tenure_train_x.columns)
importances

,Feature,Importance
2,Surname,12.774840
1,CustomerId,12.737501
0,RowNumber,12.707535
11,EstimatedSalary,12.679251
3,CreditScore,12.658319
6,Age,12.313725
7,Balance,9.515219
8,NumOfProducts,3.526369
4,Geography,3.252631
5,Gender,2.627478


In [13]:
def get_importances_columns(data, threshold):
    return data[data['Importance'] > threshold]['Feature'].values

In [14]:
columns = get_importances_columns(importances, 9)
tenure_train_x = tenure_train_x[columns]

In [15]:
tenure_valid = data_test[data_test['Tenure'].notna()] 

tenure_valid_x, tenure_valid_y = split_x_y(tenure_valid) 

tenure_valid_x = ordinal_encoder(tenure_valid_x)

        Surname Geography  Gender
7867      Yocum     Spain  Female
1402  Vorobyova    France    Male
8606   Rawlings     Spain    Male
8885    Michael     Spain    Male
6494      Pinto    France    Male
9628       Topp    France  Female
7459    Raymond     Spain  Female
3964  Sabbatini    France    Male
7000    Johnson    France    Male
9996  Johnstone    France    Male
   Surname  Geography  Gender
0   2894.0        2.0     0.0
1   2742.0        0.0     1.0
2   2240.0        2.0     1.0
3   1796.0        2.0     1.0
4   2173.0        0.0     1.0
5   2623.0        0.0     0.0
6   2241.0        2.0     0.0
7   2345.0        0.0     1.0
8   1334.0        0.0     1.0
9   1336.0        0.0     1.0


In [16]:
best_model_random_forest = None
best_accuracy = 0
best_est = 0
best_depth = 0
for est in range(1, 101, 10):
    for depth in range (1, 20):
        
        model = RandomForestClassifier(random_state=12345, n_estimators=est ,max_depth=depth)
        model.fit(tenure_train_x, tenure_train_y)
        
        predictions = model.predict(tenure_valid_x)
        result = accuracy_score(predictions, tenure_valid_y)
        
        if result > best_accuracy:
            best_model_random_forest = model
            best_accuracy = result
            best_est = est
            best_depth = depth

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Исследование задачи

## Борьба с дисбалансом

## Тестирование модели

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*